In [ ]:
!pip install torch torchvision torchaudio
!pip install diffusers transformers
!pip install flask pillow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 97.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 73.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 49.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 82.2 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [ ]:
import torch
from diffusers import StableDiffusionPipeline
from PIL import Image, ImageEnhance
import numpy as np
import os

# Step 1: Load the Stable Diffusion model
model_id = "runwayml/stable-diffusion-v1-5"
pipe = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16)
pipe = pipe.to("cuda" if torch.cuda.is_available() else "cpu")

# Step 2: Define a function to generate an image with a Ghibli-style prompt
def generate_ghibli_image(prompt, output_path="ghibli_image.png"):
    # Add Ghibli-style keywords to the prompt
    ghibli_prompt = f"{prompt}, Studio Ghibli style, soft colors, detailed background, hand-drawn aesthetic, warm lighting"

    # Generate the image
    image = pipe(ghibli_prompt, num_inference_steps=50, guidance_scale=7.5).images[0]

    # Step 3: Post-process the image to enhance the Ghibli style
    image = post_process_ghibli_style(image)

    # Save the image
    image.save(output_path)
    return image

# Step 4: Post-process the image to make it more Ghibli-like
def post_process_ghibli_style(image):
    # Convert to PIL Image if it's not already
    if not isinstance(image, Image.Image):
        image = Image.fromarray(np.uint8(image))

    # Adjust color palette (soften colors)
    enhancer = ImageEnhance.Color(image)
    image = enhancer.enhance(0.8)  # Reduce color intensity

    # Adjust brightness and contrast for a warm, soft look
    enhancer = ImageEnhance.Brightness(image)
    image = enhancer.enhance(1.1)  # Slightly increase brightness

    enhancer = ImageEnhance.Contrast(image)
    image = enhancer.enhance(0.9)  # Slightly reduce contrast

    # Apply a slight blur to mimic hand-drawn softness
    from PIL import ImageFilter
    image = image.filter(ImageFilter.GaussianBlur(radius=0.5))

    return image

# Step 5: Test the function
if __name__ == "__main__":
    prompt = "A young adventurer in a lush forest with a magical creature"
    generated_image = generate_ghibli_image(prompt)
    generated_image.show()

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

In [ ]:
def post_process_ghibli_style(image):
    # ... (other code remains the same) ...

    # Apply a slight blur to mimic hand-drawn softness
    # Changed line - Using ImageFilter.GaussianBlur for blurring
    from PIL import ImageFilter
    image = image.filter(ImageFilter.GaussianBlur(radius=0.5))

    return image

In [ ]:
from flask import Flask, request, render_template, send_file
import os

app = Flask(__name__)
UPLOAD_FOLDER = "static/uploads"
os.makedirs(UPLOAD_FOLDER, exist_ok=True)

@app.route("/", methods=["GET", "POST"])
def index():
    if request.method == "POST":
        prompt = request.form["prompt"]
        output_path = os.path.join(UPLOAD_FOLDER, "ghibli_output.png")

        # Generate the image
        generate_ghibli_image(prompt, output_path)

        return render_template("index.html", image_url=output_path)

    return render_template("index.html", image_url=None)

@app.route("/download")
def download():
    return send_file(os.path.join(UPLOAD_FOLDER, "ghibli_output.png"), as_attachment=True)

if __name__ == "__main__":
    app.run(debug=True)

 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug: * Restarting with stat


In [ ]:
from diffusers import StableDiffusionImg2ImgPipeline
from PIL import Image, ImageEnhance, ImageFilter
import torch
import numpy as np

# Load the Stable Diffusion img2img model
model_id = "runwayml/stable-diffusion-v1-5"
pipe = StableDiffusionImg2ImgPipeline.from_pretrained(model_id, torch_dtype=torch.float16)
pipe = pipe.to("cuda" if torch.cuda.is_available() else "cpu")

# Function to convert the image to Ghibli style
def convert_to_ghibli_style(input_image_path, output_image_path="ghibli_output.png", strength=0.7):
    # Load the input image
    init_image = Image.open(input_image_path).convert("RGB")
    init_image = init_image.resize((512, 512))  # Resize to match model input size

    # Generate the image
    prompt = "convert this input image to Studio Ghibli style, soft colors, detailed background, hand-drawn aesthetic, warm lighting"
    image = pipe(prompt=prompt, image=init_image, strength=strength, guidance_scale=7.5).images[0]

    # Post-process the image
    image = post_process_ghibli_style(image)

    # Save the image
    image.save(output_image_path)
    return image

# Post-processing function
def post_process_ghibli_style(image):
    if not isinstance(image, Image.Image):
        image = Image.fromarray(np.uint8(image))

    # Adjust color palette (soften colors)
    enhancer = ImageEnhance.Color(image)
    image = enhancer.enhance(0.8)  # Reduce color intensity

    # Adjust brightness and contrast for a warm, soft look
    enhancer = ImageEnhance.Brightness(image)
    image = enhancer.enhance(1.1)  # Slightly increase brightness

    enhancer = ImageEnhance.Contrast(image)
    image = enhancer.enhance(0.9)  # Slightly reduce contrast

    # Apply a slight blur to mimic hand-drawn softness
    image = image.filter(ImageFilter.GaussianBlur(radius=0.5))

    return image

# Get input and output paths from the user
input_image_path = input("input.jpeg")
output_image_path = input("output.png")

# Convert the image
converted_image = convert_to_ghibli_style(input_image_path, output_image_path)
converted_image.show()


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

input.jpeginput.jpeg
output.pngoutput.png


  0%|          | 0/35 [00:00<?, ?it/s]